# Metropolis Sampler Prototype

### Load likelihood codes and test data

In [109]:
include("/Users/nwelch/prelim/src/Julia/likelihood_functions.jl");

In [182]:
df = readtable("/Users/nwelch/prelim/data/infectionDataTest.csv");
dst = readtable("/Users/nwelch/prelim/data/infectedDistancesTest.csv");

In [183]:
head(df)

,x,y,t1,t2,t3,t4,t5,t6,tau,tauLowerBound,tauUpperBound
1,1.5,0.0,0,0,1,1,1,1,12.0,10,14
2,1.5,0.5,0,0,0,0,1,1,21.0,19,23
3,1.5,1.0,0,0,1,1,1,1,12.0,10,14
4,1.5,1.5,0,0,0,1,1,1,16.5,14,19
5,1.5,2.0,0,0,0,1,1,1,16.5,14,19
6,1.5,3.0,0,0,0,0,0,1,26.5,23,30


### Initialize priors and parameter values

In [184]:
mu_a=0.7; 
mu_b=0.004;
theta_a=0.8; 
theta_b=10;
sigma_a=0.5; 
sigma_b=100;

In [185]:
theta = theta_a*theta_b; #0.105
sigma = sigma_a*sigma_b; #1.15
mu = mu_a*mu_b; #0.003

### Simulation Setup

In [186]:
trials = 100;
infectionCount = sum(df[:t6]);
chain = Matrix(trials, (3+infectionCount));
accept = Matrix(1, (3+infectionCount));

In [187]:
chain[1,1:infectionCount] = copy(df[:tau]);
chain[1,(infectionCount+1):(infectionCount+3)] = copy([mu sigma theta]);

In [188]:
U = Uniform();

In [189]:
for r=2:trials
    
    tau = copy(chain[r-1, 1:infectionCount])
    mu = copy(chain[(r-1), (infectionCount+1)])
    sigma = copy(chain[(r-1), (infectionCount+2)])
    theta = copy(chain[(r-1), (infectionCount+3)])
    
    # sample U to generate the acceptance criteria
    u = rand(U, (infectionCount+3))
    logU = log(u)
    
    # update mu
    J_mu = Normal(mu, 0.0005)
    muStar = rand(J_mu, 1)[1]
    if muStar>0
        logLRatio_mu = llRatio_mu(mu, muStar, theta, sigma, tau, dst)
                              
        if logU[1] < logLRatio_mu
            mu = copy(muStar)
            #accept[1] = accept[1] + 1
        end
    end
  
    # update sigma
    J_sigma = Normal(sigma, 0.05)
    sigmaStar = rand(J_sigma, 1)[1]
    if sigmaStar>0.
        logLRatio_sigma = llRatio_sigma(sigma, sigmaStar,
            mu, tau, theta, dst)
        
        if logU[2] < logLRatio_sigma  
            sigma = copy(sigmaStar)
            #accept[2] = copy(accept[2]) + 1
        end
    end
    
    # update theta
    J_theta = Normal(theta, 0.005)
    thetaStar = rand(J_theta, 1)[1]
    if thetaStar>0.
        logLRatio_theta = llRatio_theta(theta, thetaStar,
            mu, tau, sigma, dst)
        
        if logU[3] < logLRatio_theta 
            theta = copy(thetaStar)
            #accept[3] = copy(accept[3]) + 1
        end 
    end
    
    for i=1:infectionCount
        
        J_taui = Normal(tau[i], 1)
        tauiStar = rand(J_taui, 1)[1]

        lowerBound = df[:tauLowerBound][i]
        upperBound = df[:tauUpperBound][i]
        
        if (tauiStar<upperBound) & (tauiStar>lowerBound)
            logLRatio_tau = llRatio_tau(tau, tauiStar, i,
                mu, theta, sigma, dst)

            if logU[(i+3)] < logLRatio_tau
                tau[i] = copy(tauiStar)
                #accept[i+3] = copy(accept[i+3]) + 1
            end
        end
    end
    chain[r,1:infectionCount] = copy(tau);
    chain[r,(infectionCount+1):(infectionCount+3)] = copy([mu sigma theta]);
end

In [180]:
chainDF = DataFrame(chain);

In [181]:
writetable("/Users/nwelch/prelim/data/mcmc_chain_julia.csv", 
    chainDF, separator = ',', header = false);